In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydotplus
from sklearn import tree
import statistics
from sklearn.metrics import accuracy_score

df = pd.read_csv('seeds.txt', sep="\t", lineterminator='\n', header = None, usecols=range(8))
df.columns = ['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove','seeds']
df_1 = df.replace(np.nan, 0, regex=True)
df_1['seeds'] = df_1['seeds'].apply(np.int64)
print(df_1)

      area  perimeter  ...  length of kernel groove  seeds
0    15.26      14.84  ...                    5.220      1
1    14.88      14.57  ...                    4.956      1
2    14.29      14.09  ...                    4.825      1
3    13.84      13.94  ...                    4.805      1
4    16.14      14.99  ...                    5.175      1
..     ...        ...  ...                      ...    ...
205  12.19      13.20  ...                    4.870      3
206  11.23      12.88  ...                    5.003      3
207  13.20      13.66  ...                    5.056      3
208  11.84      13.21  ...                    5.044      3
209  12.30      13.34  ...                    5.063      3

[210 rows x 8 columns]


In [ ]:
X = df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0) 
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8867924528301887

In [ ]:
import random
feature_cols = ['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove']
removed_rows = []
for i in range(21):
    x = random.randint(0,6)
    row = random.randint(0,209)
    if (row not in removed_rows):
        df.at[row, feature_cols[x]] = float('NaN')
        removed_rows.append(row)  

In [ ]:
dropped = df.dropna()
dropped

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,seeds
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [ ]:
df.isnull().sum()

area                       2
perimeter                  3
compactness                1
length of kernel           2
width of kernel            4
asymmetry coefficient      2
length of kernel groove    4
seeds                      0
dtype: int64

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

In [ ]:
imputer.fit(df)

KNNImputer(add_indicator=False, copy=True, metric='nan_euclidean',
           missing_values=nan, n_neighbors=5, weights='uniform')

In [ ]:
Xtrans = imputer.transform(df)

In [ ]:
predicted_vals = imputer.fit_transform(df)

In [ ]:
feature_cols = ['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove']
data_pred = pd.DataFrame(data=predicted_vals[1:,1:], index=predicted_vals[1:,0], columns=feature_cols)
data_pred.head()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove
14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1.0


In [ ]:
features = data_pred.drop(columns='length of kernel groove')
y = data_pred['length of kernel groove']

In [ ]:
feature_train, feature_test, y_train, y_test = train_test_split(features, y, test_size = 0.3)

dec_tree_ig = DecisionTreeClassifier(criterion='entropy', random_state=1)
dec_tree_ig.fit(feature_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
feature_train, feature_test, y_train, y_test = train_test_split(features, y, test_size = 0.3)

dec_tree_ig = DecisionTreeClassifier(criterion='entropy', random_state=1)
dec_tree_ig.fit(feature_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
y_pred = dec_tree_ig.predict(feature_test)

from sklearn.metrics import accuracy_score
accuracy_orig = accuracy_score(y_test, y_pred)

print ("new accuracy:", accuracy_orig*100, "%")

new accuracy: 87.3015873015873 %
